In [2]:
import pandas as pd 
import numpy as np 
import lightgbm as lgb

train = pd.read_csv('train.csv .zip')

# Unique ATMs
ATM_IDs = train.ATM_ID.unique()

def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['week'] = X['Timestamp'].dt.week
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    return X

train.columns = ['Timestamp','ForecastId','Value']
train = time_preprocess(train)

train = train.drop(train[(train['month'] == 1) | (train['month'] == 12)].index)
train = train[train.year>=2016].reset_index(drop=True)

val = train[['ForecastId','Value']].groupby('ForecastId').diff()
val['Value'] = val['Value'].fillna(0)
val['Value'] = (val['Value']==0) * 1
val.columns = ['diff']
train = pd.concat([train, val], axis=1)

a = train[['ForecastId','diff']].groupby('ForecastId').apply(pd.rolling_sum, 7, min_periods=1)
a.columns = ['ForecastId', 'to_drop']
a = a['to_drop']
train = pd.concat([train, a], axis=1)

print(train.shape)
train = train[train['to_drop']<=3].reset_index(drop=True)
train = train[['Timestamp','ForecastId','Value']]
train = time_preprocess(train)
print(train.shape)

pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

train_mask = train.Timestamp<=pd.to_datetime('2017-07-16')
valid_mask = train.Timestamp>pd.to_datetime('2017-07-16')

X_train = train[['ForecastId', 'year', 'month','week','day','week_day']][train_mask]
X_valid = train[['ForecastId', 'year', 'month','week','day','week_day']][valid_mask]
y_train = train['Value'][train_mask]
y_valid = train['Value'][valid_mask]

# custom metric 
def lgb_rmsle_score(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds
    return 'rmsle', np.sum(abs(labels-preds))/np.sum(labels)*10000, False

/home/denis/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:796: FutureWarning: pd.rolling_sum is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=7,min_periods=1,center=False).sum()
  return func(g, *args, **kwargs)


(150000, 10)
(142761, 8)
